In [1]:
import pandas as pd
import numpy as np

gene_exp = pd.read_csv('/data/keras_autoencoder_imputation/E-GEOD-78193.pcl',
                       delimiter='\t')
print(gene_exp.shape)
print(gene_exp.head())

(16240, 126)
    Gene  GSM2069305  GSM2069306  GSM2069307  GSM2069308  GSM2069309  \
0      1    0.316824    0.252532    0.243955    0.234106    0.292485   
1     10         NaN   -0.138681   -0.131388   -0.154578   -0.133985   
2    100    2.258508    2.220878    2.292011    2.221837    2.408065   
3   1000    0.015773    0.002357   -0.066824   -0.055737    0.070111   
4  10000    0.083632    0.072687    0.073163   -0.005703    0.042765   

   GSM2069310  GSM2069311  GSM2069312  GSM2069313     ...      GSM2069420  \
0    0.268240    0.321052    0.270014    0.269687     ...        0.372067   
1   -0.073763   -0.108244   -0.101171   -0.075025     ...        0.565097   
2    2.321494    2.398902    2.409410    2.167130     ...        1.911440   
3    0.058310   -0.038059   -0.017265   -0.116453     ...       -0.069461   
4    0.041141    0.025783    0.041102    0.042092     ...        0.224562   

   GSM2069421  GSM2069422  GSM2069423  GSM2069424  GSM2069425  GSM2069426  \
0    0.675189 

In [20]:
# First just replace nan with 0 (similar to denoising autoencoder)
# it may be helpful to actually add noise to other columns as well....

from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

x = gene_exp.as_matrix()[:, 1:]
x_train = np.nan_to_num(x)

input_exp = Input(shape=(125,))
encoded = Dense(64, activation='relu')(input_exp)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(125, activation='tanh')(decoded)

autoencoder = Model(input_exp, decoded)
autoencoder.compile(optimizer='adadelta', loss='mse')

autoencoder.fit(x_train, x_train,
                batch_size=256,
                epochs=100,
                shuffle=True)

x_run = x
ae_out = autoencoder.predict(x_run)

missing_mask = np.isnan(x)
x[missing_mask] = ae_out[missing_mask]

Epoch 1/100
16240/16240 [==============================] - 0s - loss: 0.4673     
Epoch 2/100
16240/16240 [==============================] - 0s - loss: 0.2347     
Epoch 3/100
16240/16240 [==============================] - 0s - loss: 0.2074     
Epoch 4/100
16240/16240 [==============================] - 0s - loss: 0.1988     
Epoch 5/100
16240/16240 [==============================] - 0s - loss: 0.1966     
Epoch 6/100
16240/16240 [==============================] - 0s - loss: 0.1957     
Epoch 7/100
16240/16240 [==============================] - 0s - loss: 0.1950     
Epoch 8/100
16240/16240 [==============================] - 0s - loss: 0.1946     
Epoch 9/100
16240/16240 [==============================] - 0s - loss: 0.1942     
Epoch 10/100
16240/16240 [==============================] - 0s - loss: 0.1939     
Epoch 11/100
16240/16240 [==============================] - 0s - loss: 0.1936     
Epoch 12/100
16240/16240 [==============================] - 0s - loss: 0.1934     
Epoch 13/100


In [37]:
# test with data MCAR
md_per = 0.02

x = gene_exp.as_matrix()[:, 1:]
orig_present = ~np.isnan(x)

for row in x:
    rand_missing = np.random.randint(0, x.shape[1], int(np.ceil(x.shape[1]*md_per)))
    row[rand_missing] = np.nan
    
x_train = np.nan_to_num(x)
missing_mask = np.isnan(x)

input_exp = Input(shape=(125,))
encoded = Dense(64, activation='relu')(input_exp)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(125, activation='tanh')(decoded)

autoencoder = Model(input_exp, decoded)
autoencoder.compile(optimizer='adadelta', loss='mse')

# first run for 2 epochs should be not great fit
autoencoder.fit(x_train, x_train,
                batch_size=256,
                epochs=2,
                shuffle=True)

x_run = x
x1 = x
ae_out_1 = autoencoder.predict(x_run)
x1[missing_mask] = ae_out_1[missing_mask]

print('MSE after 2 epochs:')
print(((x1[orig_present] - gene_exp.as_matrix()[:, 1:][orig_present]) ** 2).mean(axis=None))
print('')

# now run for 18 more
autoencoder.fit(x_train, x_train,
                batch_size=256,
                epochs=18,
                shuffle=True)

x2 = x
ae_out_2 = autoencoder.predict(x_run)
x2[missing_mask] = ae_out_2[missing_mask]

print('MSE after 20 epochs:')
print(((x1[orig_present] - gene_exp.as_matrix()[:, 1:][orig_present]) ** 2).mean(axis=None))
print('')

Epoch 1/2
16240/16240 [==============================] - 0s - loss: 0.4803     
Epoch 2/2
16240/16240 [==============================] - 0s - loss: 0.2413     
MSE after 2 epochs:
0.0200173653288

Epoch 1/18
16240/16240 [==============================] - 0s - loss: 0.2101     
Epoch 2/18
16240/16240 [==============================] - 0s - loss: 0.2029     
Epoch 3/18
16240/16240 [==============================] - 0s - loss: 0.2013     
Epoch 4/18
16240/16240 [==============================] - 0s - loss: 0.2003     
Epoch 5/18
16240/16240 [==============================] - 0s - loss: 0.1997     
Epoch 6/18
16240/16240 [==============================] - 0s - loss: 0.1992     
Epoch 7/18
16240/16240 [==============================] - 0s - loss: 0.1988     
Epoch 8/18
16240/16240 [==============================] - 0s - loss: 0.1985     
Epoch 9/18
16240/16240 [==============================] - 0s - loss: 0.1982     
Epoch 10/18
16240/16240 [==============================] - 0s - loss: 0.19